NOTES/TODO:
1. How do we handle different scoring scales?
2. How do we handle essay set 2's "dual-score" method? (Scored across 2 different domains)
3. Which crieterion do we use for evaluating our classifier?  I vote kappa.

In [1]:
import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

# TODO if needed, include the words and stopwords imports
# HOWEVER, to use them, you will need to download nltk stuff first if not done already
# To do so, open a python shell (i.e. go to terminal and enter python), and then type
#
# import nltk
# nltk.download()

# After this, select the words and stopwords corpuses, and download them

#import nltk
#from nltk.corpus import stopwords
#from nltk.corpus import words

# Regular expressions might be useful
import re

# Beautiful soup might be useful
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

In [2]:
# Read in training data
# TODO For now, only includes domain1_score, i.e. ignores essay set 2's dual-score system
train_cols = ['essay_id', 'essay_set', 'essay', 'domain1_score']
train_df = pd.read_csv('data/training_set_rel3.tsv', delimiter='\t', usecols=train_cols)
print train_df.head()

   essay_id  essay_set                                              essay  \
0         1          1  Dear local newspaper, I think effects computer...   
1         2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2         3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3         4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4         5          1  Dear @LOCATION1, I know having computers has a...   

   domain1_score  
0              8  
1              9  
2              7  
3             10  
4              8  


In [3]:
# Show nothing is empty in training set
if train_df.isnull().any().any():
    print 'Data is missing!'
else:
    print 'No missing data!'

No missing data!


In [4]:
# Read in validation data
# TODO For now, only includes domain1_predictionid, i.e. ignores essay set 2's dual-score system
valid_cols = ['essay_id', 'essay_set', 'essay', 'domain1_predictionid']
valid_df = pd.read_csv('data/valid_set.tsv', delimiter='\t', usecols=valid_cols)
print valid_df.head()

   essay_id  essay_set                                              essay  \
0      1788          1  Dear @ORGANIZATION1, @CAPS1 more and more peop...   
1      1789          1  Dear @LOCATION1 Time @CAPS1 me tell you what I...   
2      1790          1  Dear Local newspaper, Have you been spending a...   
3      1791          1  Dear Readers, @CAPS1 you imagine how life woul...   
4      1792          1  Dear newspaper, I strongly believe that comput...   

   domain1_predictionid  
0                  1788  
1                  1789  
2                  1790  
3                  1791  
4                  1792  


In [5]:
# Show nothing is empty in validation set
if valid_df.isnull().any().any():
    print 'Data is missing!'
else:
    print 'No missing data!'

No missing data!


In [6]:
# maps validations set domain1_predictionid to a score
# note that scoring system is different for each essay set
valid_scores = pd.read_csv('data/valid_sample_submission_2_column.csv', delimiter=',')
print valid_scores.head()

   prediction_id  predicted_score
0           1788                7
1           1789                8
2           1790                9
3           1791                9
4           1792                9


In [7]:
# Show nothing is empty in validation scores
if valid_scores.isnull().any().any():
    print 'Data is missing!'
else:
    print 'No missing data!'

No missing data!


In [21]:
# returned a copy of old_df, with essays cleaned for count vectorizer
def vectorizer_clean(old_df):
    new_df = old_df.copy()
    for i in xrange(new_df.shape[0]):
        new_df.set_value(i, 'essay', " ".join(re.sub('[^a-zA-Z\d\s]', '', new_df['essay'].iloc[i]).lower().split())) 
    return new_df

In [22]:
# print essays cleaned for vectorizer (essay is now just lowercase words separated by space) 
vectorizer_train = vectorizer_clean(train_df)
print vectorizer_train.head()

   essay_id  essay_set                                              essay  \
0         1          1  dear local newspaper i think effects computers...   
1         2          1  dear caps1 caps2 i believe that using computer...   
2         3          1  dear caps1 caps2 caps3 more and more people us...   
3         4          1  dear local newspaper caps1 i have found that m...   
4         5          1  dear location1 i know having computers has a p...   

   domain1_score  
0              8  
1              9  
2              7  
3             10  
4              8  
